This Python Jupyter Notebook can be run on colab.research.google.com. According to this [Github Issue(https://github.com/tensorflow/tensorflow/issues/32043)] it should work once Tensorflow 2.1 is available on Colab.

In [0]:
%tensorflow_version 2.x



TensorFlow 2.x selected.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Try to access the `rpml` folder with `Data/Training_Data_and_Labels` and (empty) `Data/NNResults` subfolders on your Google Drive and change `root_dir` to the path of `rpml`.

In [0]:
!ls /content/drive/My\ Drive/

ls: cannot access '/content/drive/My Drive/rpml1/': No such file or directory


In [ ]:


"""
Runs neural network on data in folder 'Data/Training_Data_and_labels'

Adapted from 'main.py' on 2019-08-14 by Robert Fischbach.
Uses parameters guessed manually, without using Talos.
"""
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.backend as K
from sklearn.utils import shuffle
import os

# ************************* Config *************************************
# Change this to the directory of the local git repository.
root_dir = '/content/drive/My Drive/rpml/'
# Control numerical precision, can be 'float16', 'float32', or 'float64'
floatx_wanted = 'float32'
K.set_floatx(floatx_wanted)
# Choose wether to normalize data to the mean and standard deviation of the
# training data. Not clear if it improves or hampers performance.
normalize_data = True
# Use a unique name for each experiment so that the results are saved separately
experiment_name = '1500epochs'
# ************************* Config *************************************


def train_input_fn(batch_size=1024):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
# Shuffle, repeat, and batch the examples.
    dataset = dataset.cache()
    dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size, drop_remainder=True)
# Return the dataset.
    return dataset


def short_model(x_train, y_train, x_val, y_val):
    """Run a Keras sequential Neural Network.

    Args:
        x_train (numpy array):                  Training features
        y_train (numpy array):                  Training labels
        x_val (numpy array):                    Validation features
        y_val (numpy array):                    Validation labels
        metrics_cb (inst of Keras Callback()):  Callback metric

    Returns:
        history [keras.callbacks.History]: Data about training history
        model   [keras.models.Sequential]: Fitted Neural Network
    """
    
    cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_host(cluster_resolver.master())
    tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(cluster_resolver)
    
# The commented out version works with Tensorflow 1.x, but does not support 
# `class_weight`
#     resolver = tf.contrib.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
#     tf.contrib.distribute.initialize_tpu_system(resolver)
#     strategy = tf.contrib.distribute.TPUStrategy(resolver)
#   with strategy.scope():    
    with tpu_strategy.scope():
      model = Sequential()
      model.add(Dense(16, input_shape=(x_train.shape[1],), activation='relu'))
      model.add(Dense(16, activation='relu'))
      model.add(Dense(1, activation='sigmoid'))
      model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.25),
                    loss='binary_crossentropy',
                    metrics=['acc', 'binary_crossentropy'])

    history = model.fit(x_train, y_train,
                        batch_size= 20000,   #int(y_train.shape[0] * 0.1),
                        epochs=1500,
                        validation_data=[x_val, y_val],
                        verbose=1,
                        class_weight={0:1.0, 1:25.0}
                        )
    return history, model


def convert_to_bools(nparray):
    """Replaces all above-one values in a list of integer values with '1'

    Args:
        nparray:   Numpy array

    Returns:
        binary_nparray:    Numpy array consisting of '0's and '1's
    """
    ones = np.ones(np.shape(nparray))
    binary_nparray = np.minimum(nparray, ones)
    return binary_nparray


def main(root_dir):
    """Run neural networks with imported feature and label data and save them.

    Args:
        root_dir (string):      directory with the Data/NNResults and
                                Data/Training_Data_and_Labels folders

    """

    # Leave out  mgc_fft_2, like Tabrizi 2018, to have a 'clean' test case.
    benchmark_names = ['mgc_des_perf_1', 'mgc_des_perf_a', 'mgc_des_perf_b',
                    'mgc_fft_1', 'mgc_fft_a',
                    'mgc_matrix_mult_1', 'mgc_matrix_mult_a',
                    'mgc_pci_bridge32_a', 'mgc_pci_bridge32_b']
    features = np.zeros((0, 9, 8))
    labels = np.zeros((0,))
    for i in range(len(benchmark_names)):
        benchmark_name = benchmark_names[i]
        folder_prefix_training = root_dir + 'Data/Training_Data_and_Labels/design_placed_training_data_'
        folder_prefix_labels = root_dir + 'Data/Training_Data_and_Labels/design_routed_labels_shorts_'

        train_to_validation_ratio = 0.8
        assert(0 < train_to_validation_ratio < 1)

        fh = open(folder_prefix_training + benchmark_name + '.pickle', 'br')
        features_cur = pickle.load(fh)
        fh.close()

        fh = open(folder_prefix_labels + benchmark_name + '.pickle', 'br')
        labels_cur = pickle.load(fh)
        fh.close()

        labels_cur = np.asarray(labels_cur)
        features_cur = np.asarray(features_cur)

        # Convert numbers of shorts to binary 0s and 1s (exist or not).
        labels_cur = convert_to_bools(labels_cur)
        print(labels_cur)
        labels_cur = np.asarray(labels_cur)

        print('labels.shape', labels.shape)
        print('labels_cur.shape', labels_cur.shape)
        print('features_cur.shape', features_cur.shape)
        print('features.shape', features.shape)
        labels = np.concatenate((labels, labels_cur), axis=0)
        features = np.concatenate((features, features_cur), axis=0)
    assert(len(features) == len(labels))
    print('labels.shape:', labels.shape)
    print('features.shape:', features.shape)
    benchmark_name = 'combined_without_mgc_fft_2'

    print('\nFitting model for benchmark', benchmark_name)

    x, y = shuffle(features, labels, random_state=666)
    x = np.array(x)

    x = x.reshape(x.shape[0], x.shape[1] * x.shape[2])
    y = np.array(y)
    print('Total number of shorts:', np.sum(y))
    print('x.shape:', x.shape)
    print('y.shape:', y.shape)
    
    num_training_samples = int(len(x) * train_to_validation_ratio)
    x_train = x[:num_training_samples]
    y_train = y[:num_training_samples]
    x_val = x[num_training_samples:]
    y_val = y[num_training_samples:]

    if normalize_data:
        # Normalize Data as in 'Deep Learning with Python', p.86
        mean = x_train.mean(axis=0)
        x_train -= mean
        std = x_train.std(axis=0)
        x_train /= std
        # Normalize x_val with mean and std from x_train to prevent data leakage
        x_val -= mean
        x_val /= std
    
    history, model = short_model(x_train, y_train, x_val, y_val)

    # Save history dictionary and model
    fname_history = root_dir + 'Data/NNResults/' + benchmark_name + '_' + \
                    experiment_name + '_history.pickle'
    with open(fname_history, 'wb') as f:
        pickle.dump(history.history, f, pickle.DEFAULT_PROTOCOL)

    fname_model = root_dir + 'Data/NNResults/' + benchmark_name + '_' + \
                  experiment_name + '_model.h5' 
    model.save(fname_model)
    

# Do not execute main() when imported as module
if __name__ == '__main__':
    main(root_dir)